#Pneumonia Detection using Transfer Learning

In [1]:
# Installing the required modules
! pip install -q kaggle


In [22]:
! pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


In [2]:
!pip install tensorflow
!pip install keras
!pip install scipy
!pip install glob2

In [3]:
# Downloading Dataset from Kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [01:49<00:00, 23.3MB/s]
100% 2.29G/2.29G [01:49<00:00, 22.5MB/s]


In [6]:
# Unzip dataset
!unzip -q chest-xray-pneumonia.zip

In [7]:
pwd

'/content'

In [8]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob


In [9]:
IMAGESHAPE = [224, 224]
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test'


In [10]:
vgg_model = VGG16(input_shape=IMAGESHAPE + [3], weights='imagenet', include_top=False)


58889256/58889256 [==============================] - 4s 0us/step


In [11]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False


In [12]:
classes = glob('chest_xray/train/*')


In [13]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)


In [14]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [16]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)


In [17]:
training_set = train_datagen.flow_from_directory('chest_xray/train',
												target_size = (224, 224),
												batch_size = 4,
												class_mode = 'categorical')


Found 5216 images belonging to 2 classes.


In [18]:
test_set = testing_datagen.flow_from_directory('chest_xray/test',
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'categorical')


Found 624 images belonging to 2 classes.


In [19]:
fitted_model = final_model.fit(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)



Epoch 1/5
1304/1304 [==============================] - 127s 94ms/step - loss: 0.2417 - accuracy: 0.9264 - val_loss: 0.4768 - val_accuracy: 0.9087
Epoch 2/5
1304/1304 [==============================] - 122s 94ms/step - loss: 0.2798 - accuracy: 0.9406 - val_loss: 0.9110 - val_accuracy: 0.9006
Epoch 3/5
1304/1304 [==============================] - 122s 94ms/step - loss: 0.2441 - accuracy: 0.9511 - val_loss: 0.7343 - val_accuracy: 0.9038
Epoch 4/5
1304/1304 [==============================] - 126s 96ms/step - loss: 0.2674 - accuracy: 0.9546 - val_loss: 0.8477 - val_accuracy: 0.9167
Epoch 5/5
1304/1304 [==============================] - 125s 96ms/step - loss: 0.1933 - accuracy: 0.9647 - val_loss: 0.9819 - val_accuracy: 0.8910


In [20]:
#Saving Model
final_model.save('our_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
# Testing
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('our_model.h5') #Loading our model
img=image.load_img('/content/pneumonia.jpg',target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
else:
	print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')


1/1 [==============================] - 1s 1s/step
Person is affected with Pneumonia.
Predictions: [[0. 1.]]
